In [1]:
import numpy as np
import csv
import math
import progressbar
import xgboostlib

In [2]:
# load csv file
data_pathname = "/Users/Jason/Documents/CS 221/project/dataset/ASR2016-clean.csv"
data_file = open(data_pathname, 'r')
reader = csv.reader(data_file)
headers = next(reader, None) # skip headers in csv

# northeast, south, midwest, west
X = [[], [], [], []] 
y = [[], [], [], []]

# load regional data and labels
for row in reader:
    row = [float(item) if item !=' ' else -1.0 for item in row]
    
    region = int(row[0] - 1)
    
    label = row[-1] == float('1') # is employed
    features = row[1:-1] # 32 features
    
    X[region].append(features)
    y[region].append(label)

In [3]:
# split regions into train-test-validation sets
NUM_REGIONS = 4
PERCENT_TEST = 0.8

X_train = [[], [], [], []] 
X_test = [[], [], [], []] 
y_train = [[], [], [], []]
y_test = [[], [], [], []]

for region in range(NUM_REGIONS):
    num_training = int(PERCENT_TEST*len(y[region]))
    
    X_train[region] = np.asarray(X[region])[:num_training]
    X_test[region] = np.asarray(X[region])[num_training + 1:]
    
    y_train[region] = np.asarray(y[region])[:num_training]
    y_test[region] = np.asarray(y[region])[num_training + 1:]

In [4]:
# XGBoost Classifier (from https://github.com/eriklindernoren/ML-From-Scratch)
# train XGBoost neighbors classification on each region 
clfs = [[], [], [], []]
for region in range(NUM_REGIONS):
    y_train[region] = y_train[region].astype(int)
    clfs[region] = xgboostlib.XGBoost()
    clfs[region].fit(X_train[region], y_train[region])

KeyboardInterrupt: 

In [ ]:
# evaluate XGBoost neighbors on each region in test set
for region in range(NUM_REGIONS):
    y_pred = clfs[region].predict(X_test[region])
    accuracy = accuracy_score(y_test[region], y_pred)
    print(accuracy)